In [ ]:
import os
import h5py
import shutil
from astropy.nddata.utils import Cutout2D
from astropy.io import fits
from astropy import table
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import (ZScaleInterval, ImageNormalize)

# Initialize hdf5 file

In [ ]:
os.path.expandvars("$SLURM_TMPDIR")

In [ ]:
hf = h5py.File("cutouts.h5", "w")
hf.close()
src = os.path.expandvars("$SCRATCH") + "/cutouts.h5"
dest = os.path.expandvars("$SLURM_TMPDIR") + "/"
shutil.copy2(src, dest)

In [ ]:
hf = h5py.File(dest + "cutouts.h5", "r+")

In [ ]:
image_dir = "/home/eyvorch9/projects/rrg-kyi/astro/cfis/W3/"

In [ ]:
# Only use tiles with all five channels

tile_list = open(image_dir + "tiles.list", "r")

u_images = []
u_weights = []
g_images = []
g_weights = []
r_images = []
r_weights = []
i_images= []
i_weights = []
z_images = []
z_weights = []
cats = []

for tile in tile_list:
    tile = tile[:-1] # Remove new line character
    channels = tile.split(" ")
    if len(channels) == 5: # Order is u,g,r,i,z
        u_images.append(image_dir + channels[0] + ".fits")
        u_weights.append(image_dir + channels[0] + ".weight.fits.fz")
        g_images.append(image_dir + channels[1] + ".fits")
        g_weights.append(image_dir + channels[1] + ".wt.fits")
        r_images.append(image_dir + channels[2] + ".fits")
        r_weights.append(image_dir + channels[2] + ".weight.fits.fz")
        i_images.append(image_dir + channels[3] + ".fits")
        i_weights.append(image_dir + channels[3] + ".wt.fits")
        z_images.append(image_dir + channels[4] + ".fits")
        z_weights.append(image_dir + channels[4] + ".wt.fits")
        cats.append(image_dir + channels[2] + ".cat")
tile_list.close()

In [ ]:
print(len(u_images))
print(len(u_weights))
print(len(g_images))
print(len(g_weights))
print(len(r_images))
print(len(r_weights))
print(len(i_images))
print(len(i_weights))
print(len(z_images))
print(len(z_weights))
print(len(cats))

In [ ]:
# Copy first 20 tiles to $SLURM_TMPDIR
n_tiles = 20
for n in range(n_tiles):
    shutil.copy2(u_images[n], dest)
    shutil.copy2(u_weights[n], dest)
    shutil.copy2(g_images[n], dest)
    shutil.copy2(g_weights[n], dest)
    shutil.copy2(r_images[n], dest)
    shutil.copy2(r_weights[n], dest)
    shutil.copy2(i_images[n], dest)
    shutil.copy2(i_weights[n], dest)
    shutil.copy2(z_images[n], dest)
    shutil.copy2(z_weights[n], dest)
    shutil.copy2(cats[n], dest)
    
    u_images[n] = os.path.abspath(dest + os.path.basename(u_images[n]))
    u_weights[n] = os.path.abspath(dest + os.path.basename(u_weights[n]))
    g_images[n] = os.path.abspath(dest + os.path.basename(g_images[n]))
    g_weights[n] = os.path.abspath(dest + os.path.basename(g_weights[n]))
    r_images[n] = os.path.abspath(dest + os.path.basename(r_images[n]))
    r_weights[n] = os.path.abspath(dest + os.path.basename(r_weights[n]))
    i_images[n] = os.path.abspath(dest + os.path.basename(i_images[n]))
    i_weights[n] = os.path.abspath(dest + os.path.basename(i_weights[n]))
    z_images[n] = os.path.abspath(dest + os.path.basename(z_images[n]))
    z_weights[n] = os.path.abspath(dest + os.path.basename(z_weights[n]))    
    cats[n] = os.path.abspath(dest + os.path.basename(cats[n]))    

# Store cutouts into hdf5 file

In [ ]:
def create_cutouts(img, wt, x, y):
    img_cutout = Cutout2D(img.data, (x, y), cutout_size, mode="partial", fill_value=0).data
    wt_cutout = Cutout2D(wt.data, (x, y), cutout_size, mode="partial", fill_value=0).data
    img_cutout[np.isnan(img_cutout)] = 0
    wt_cutout[np.isnan(wt_cutout)] = 0
    
    lower = np.percentile(img_cutout, 1)
    upper = np.percentile(img_cutout, 99)
    if upper == lower: # Avoid division by 0
        img_norm = img_cutout
        wt_norm = wt_cutout
    else:
        img_norm = (img_cutout - np.min(img_cutout)) / (upper - lower)
        wt_norm = (wt_cutout - np.min(wt_cutout)) / (upper - lower)
    
    return (img_norm, wt_norm)

In [ ]:
cutout_size = 64
img_cutout = np.zeros((cutout_size, cutout_size, 5))
wt_cutout = np.zeros((cutout_size, cutout_size, 5))
for n in range(n_tiles):
    root = os.path.basename(u_images[n])[5:12] + "/" # XXX.XXX id
    
    u_image = fits.open(u_images[n], memmap=True)
    u_weight = fits.open(u_weights[n], memmap=True)
    g_image = fits.open(g_images[n], memmap=True)
    g_weight = fits.open(g_weights[n], memmap=True)
    r_image = fits.open(r_images[n], memmap=True)
    r_weight = fits.open(r_weights[n], memmap=True)
    i_image = fits.open(i_images[n], memmap=True)
    i_weight = fits.open(i_weights[n], memmap=True)
    z_image = fits.open(z_images[n], memmap=True)
    z_weight = fits.open(z_weights[n], memmap=True)
    cat = table.Table.read(cats[n], format="ascii.sextractor")
    
    img_group = hf.create_group(root + "IMAGES")
    wt_group = hf.create_group(root + "WEIGHTS")

    for i, (x, y) in enumerate(zip(cat["X_IMAGE"], cat["Y_IMAGE"])):
        (u_img, u_wt) = create_cutouts(u_image[0], u_weight[1], x, y)
        (g_img, g_wt) = create_cutouts(g_image[0], g_weight[0], x, y)
        (r_img, r_wt) = create_cutouts(r_image[0], r_weight[1], x, y)
        (i_img, i_wt) = create_cutouts(i_image[0], i_weight[0], x, y)
        (z_img, z_wt) = create_cutouts(z_image[0], z_weight[0], x, y)
        
        img_cutout[:,:,0] = u_img
        img_cutout[:,:,1] = g_img
        img_cutout[:,:,2] = r_img
        img_cutout[:,:,3] = i_img
        img_cutout[:,:,4] = z_img
        wt_cutout[:,:,0] = u_wt
        wt_cutout[:,:,1] = g_wt
        wt_cutout[:,:,2] = r_wt
        wt_cutout[:,:,3] = i_wt
        wt_cutout[:,:,4] = z_wt

        img_group.create_dataset(f"c{i}", data=img_cutout)
        wt_group.create_dataset(f"c{i}", data=wt_cutout)
    
    u_image.close()
    u_weight.close()
    g_image.close()
    g_weight.close()
    r_image.close()
    r_weight.close()
    i_image.close()
    i_weight.close()
    z_image.close()
    z_weight.close()
    print(f"Tile {n} completed")

# Test getting and plotting cutouts from hdf5 file

In [ ]:
group = hf.get("180.272/IMAGES")
plot_cutouts = np.array(group.get("c4"))
print(plot_cutouts.shape)

In [ ]:
channels = ["u", "g", "r", "i", "z"]
fig, axes = plt.subplots(1,5, figsize=(12,8))
for i in range(5):
    axes[i].imshow(plot_cutouts[:,:,i])
    axes[i].set_title(channels[i])

In [ ]:
hf.close()
src = os.path.expandvars("$SLURM_TMPDIR") + "/cutouts.h5"
dest = os.path.expandvars("$SCRATCH") + "/"
shutil.copy2(src, dest)